### 1. Librerias 

In [181]:
import time
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
import mysql.connector
from mysql.connector import errorcode
import os

### 2. Función para extraer los datos(Web scarping)

In [182]:
def extract_text(id, url, by_type, identifier):
    # Configurar el servicio para ChromeDriver
    service = Service(ChromeDriverManager().install())

    # Iniciar el navegador
    driver = webdriver.Chrome(service=service)

    try:
        # Navegar a la página web
        print(f"Navegando a la URL: {url}")
        driver.get(url)
        
        time.sleep(2)# pausa de 2 segundos
        # Hacer scroll hacia abajo para cargar el contenido si es necesario
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)
        
        # Esperar hasta que el contenedor de los productos esté presente
        wait = WebDriverWait(driver, 5)  # Incrementar el tiempo de espera
        print(f"Esperando el elemento con {by_type} y {identifier}")
        wait.until(EC.presence_of_element_located((by_type, identifier)))

        # Extraer datos utilizando el tipo de localización y el identificador proporcionado
        element = driver.find_element(by_type, identifier)
        print(f"Elemento encontrado: {element}")

        # Obtener el texto del elemento
        text = element.text
        print(f"Texto encontrado para {id}: {text}")
        text_name = (id, text)
        
        return text_name
    except Exception as e:
        print(f"Error al extraer texto para {id} con URL {url}: {e}")
        return None
    finally:
        # Asegurarse de que el navegador se cierra
        driver.quit()





### 3. Creación de los argumentos que se van a pasar a la función

In [183]:
# id, empresa, producto, url, xpath
# links_products = [
#       ['1', 'Carrefour', 'azucar', 'https://www.carrefour.com.ar/azucar-comun-tipo-a-bella-vista-1-kg-421740/p', '/html/body/div[3]/div/div[1]/div/div/div/div[5]/div/div[2]/div[3]/section/div/div[2]/div/div[10]/div/div/div/div/div/div[2]/div/div/div[1]/div/div[1]/span/span/span[1]'],
#       ['2', 'Dia', 'azucar', 'https://diaonline.supermercadosdia.com.ar/az%C3%BAcares?_q=az%C3%BAcares&map=ft', '/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div[3]/section/div/div[3]/div/div[3]/div/div/div/div/div[5]/section/a/article/div/div/div[5]/div/div/div[1]/span/span/span'],
#       ['3', 'Disco', 'azucar', 'https://www.disco.com.ar/azucar?_q=azucar&map=ft&page=1', '/html/body/div[2]/div/div[1]/div/div[10]/div/div[2]/section/div[2]/div/div[4]/section/div/div/div/div/div[2]/div/div[3]/div/div/div/div/div[1]/section/a/article/div[5]/div/div/div/div[1]/div/span/div/div'],
#       ['4', 'Carrefour', 'arroz', 'https://www.carrefour.com.ar/arroz-molinos-ala-parboil-bolsa-1-kg/p', '/html/body/div[3]/div/div[1]/div/div/div/div[5]/div/div[2]/div[3]/section/div/div[2]/div/div[10]/div/div/div/div/div/div[2]/div/div/div[1]/div/div[1]/span/span/span'],
#       ['5', 'Dia', 'arroz', 'https://diaonline.supermercadosdia.com.ar/arroz-parboil-molinos-ala-1-kg-33019/p', '/html/body/div[2]/div/div[1]/div/div/div/div[3]/div/div[2]/div/section/div/div[2]/div/div[5]/div/div/div[1]/div/div/span/span/span'],
#       ['6', 'Disco', 'arroz', 'https://www.disco.com.ar/arroz?_q=arroz&map=ft', '/html/body/div[2]/div/div[1]/div/div[10]/div/div[2]/section/div[2]/div/div[4]/section/div/div/div/div/div[2]/div/div[3]/div/div/div/div/div[4]/section/a/article/div[5]/div/div/div/div[1]/div/span/div/div'],
#       ['7', 'Carrefour', 'aceite', 'https://www.carrefour.com.ar/aceite-de-girasol-natura-900-cc/p','/html/body/div[3]/div/div[1]/div/div/div/div[5]/div/div[2]/div[3]/section/div/div[2]/div/div[10]/div/div/div/div/div/div[2]/div/div/div[1]'],
#       ['8', 'Dia', 'aceite', 'https://diaonline.supermercadosdia.com.ar/aceite-girasol-natural-09-l-78855/p', '/html/body/div[2]/div/div[1]/div/div/div/div[3]/div/div[2]/div/section/div/div[2]/div/div[5]/div/div/div[1]/div/div'],
#       ['9', 'Disco', 'aceite', 'https://www.disco.com.ar/aceite?_q=aceite&map=ft', '/html/body/div[2]/div/div[1]/div/div[10]/div/div[2]/section/div[2]/div/div[4]/section/div/div/div/div/div[2]/div/div[3]/div/div/div/div/div[3]/section/a/article/div[5]/div/div/div/div[1]/div/span/div/div'],
#       ['10', 'Carrefour', 'leche','https://www.carrefour.com.ar/Lacteos-y-productos-frescos/Leches?order=', '/html/body/div[3]/div/div[1]/div/div[5]/div/div/section/div[2]/div/div[3]/div/div[2]/div/div[3]/div/div/div/div[2]/div[1]/section/a/article/div/div[10]/div/div/div/div[1]/span/span/span[1]'],
#       ['11', 'Dia', 'leche', 'https://diaonline.supermercadosdia.com.ar/leche?_q=leche&map=ft', '/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div[3]/section/div/div[3]/div/div[3]/div/div/div/div/div[6]/section/a/article/div/div/div[5]/div/div/div[1]/span/span/span'],
#       ['12', 'Disco', 'leche', 'https://www.disco.com.ar/leche?_q=leche&map=ft', '/html/body/div[2]/div/div[1]/div/div[10]/div/div[2]/section/div[2]/div/div[4]/section/div/div/div/div/div[2]/div/div[3]/div/div/div/div/div[6]/section/a/article/div[5]/div/div/div/div[1]/div/span/div/div']
# ]fideos, huevos, queso, pan
links_products = [
        ['13', 'Carrefour', 'fideos', 'https://www.carrefour.com.ar/Almacen/Pastas-secas?order=','/html/body/div[3]/div/div[1]/div/div[5]/div/div/section/div[2]/div/div[3]/div/div[2]/div/div[3]/div/div/div/div[2]/div[14]/section/a/article/div/div[10]/div/div/div/div[1]/span/span/span'],
        ['14', 'Dia', 'fideos', 'https://diaonline.supermercadosdia.com.ar/fideos?_q=fideos&map=ft', '/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div[3]/section/div/div[3]/div/div[3]/div/div/div/div/div[14]/section/a/article/div/div/div[5]/div/div/div[1]/span/span/span'],
        ['15', 'Disco', 'fideos', 'https://www.disco.com.ar/fideos?_q=fideos&map=ft', '/html/body/div[2]/div/div[1]/div/div[10]/div/div[2]/section/div[2]/div/div[4]/section/div/div/div/div/div[2]/div/div[3]/div/div/div/div/div[2]/section/a/article/div[5]/div/div/div/div[1]/div/span/div/div'],
        ['16', 'Carrefour', 'huevos', 'https://www.carrefour.com.ar/huevos?_q=huevos&map=ft', '/html/body/div[3]/div/div[1]/div/div[5]/div/div/section/div[2]/div/div[3]/div/div[2]/div/div[3]/div/div/div/div[2]/div[1]/section/a/article/div/div[10]/div/div/div/div[2]/span/span/span'],
        ['17', 'Dia', 'huevos', 'https://diaonline.supermercadosdia.com.ar/huevos?_q=huevos&map=ft&page=2', '/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div[3]/section/div/div[3]/div/div[3]/div/div/div/div/div[4]/section/a/article/div/div/div[5]/div/div/div[1]/span/span/span'],
        ['18', 'Disco', 'huevos', 'https://www.disco.com.ar/huevos?_q=huevos&map=ft', '/html/body/div[2]/div/div[1]/div/div[10]/div/div[2]/section/div[2]/div/div[4]/section/div/div/div/div/div[2]/div/div[3]/div/div/div/div/div[8]/section/a/article/div[5]/div/div/div/div[1]/div/span/div/div'],
        ['19', 'Carrefour', 'queso', 'https://www.carrefour.com.ar/Lacteos-y-productos-frescos/Quesos?order=&page=3', '/html/body/div[3]/div/div[1]/div/div[5]/div/div/section/div[2]/div/div[3]/div/div[2]/div/div[3]/div/div/div/div[2]/div[11]/section/a/article/div/div[10]/div/div/div/div[1]/span/span/span'],
        ['20', 'Dia', 'queso', 'https://diaonline.supermercadosdia.com.ar/quesos?_q=quesos&map=ft', '/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div[3]/section/div/div[3]/div/div[3]/div/div/div/div/div[3]/section/a/article/div/div/div[5]/div/div/div[1]/span/span'],
        ['21', 'Disco', 'queso', 'https://www.disco.com.ar/quesos?_q=quesos&map=ft', '/html/body/div[2]/div/div[1]/div/div[10]/div/div[2]/section/div[2]/div/div[4]/section/div/div/div/div/div[2]/div/div[3]/div/div/div/div/div[1]/section/a/article/div[5]/div/div/div/div[1]/div/span/div/div'],
        ['22', 'Carrefour', 'pan lactal', 'https://www.carrefour.com.ar/Panaderia/Panificados?order=&page=1', '/html/body/div[3]/div/div[1]/div/div[5]/div/div/section/div[2]/div/div[3]/div/div[2]/div/div[3]/div/div/div/div[2]/div[4]/section/a/article/div/div[10]/div/div/div/div[1]/span/span/span'],
        ['23', 'Dia','pan lactal', 'https://diaonline.supermercadosdia.com.ar/pan?_q=pan&map=ft', '/html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div[3]/section/div/div[3]/div/div[3]/div/div/div/div/div[1]/section/a/article/div/div/div[5]/div/div/div[1]/span/span/span'],
        ['24', 'Disco','pan lactal', 'https://www.disco.com.ar/pan?_q=pan&map=ft', '/html/body/div[2]/div/div[1]/div/div[10]/div/div[2]/section/div[2]/div/div[4]/section/div/div/div/div/div[2]/div/div[3]/div/div/div/div/div[2]/section/a/article/div[5]/div/div/div/div[1]/div/span/div/div']
]

In [184]:
# Se crea un dataframe con la lista links_products
columnas = ['id','empresa', 'producto','link', 'xpath']
df = pd.DataFrame(links_products, columns=columnas)
df


,id,empresa,producto,link,xpath
0,13,Carrefour,fideos,https://www.carrefour.com.ar/Almacen/Pastas-se...,/html/body/div[3]/div/div[1]/div/div[5]/div/di...
1,14,Dia,fideos,https://diaonline.supermercadosdia.com.ar/fide...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
2,15,Disco,fideos,https://www.disco.com.ar/fideos?_q=fideos&map=ft,/html/body/div[2]/div/div[1]/div/div[10]/div/d...
3,16,Carrefour,huevos,https://www.carrefour.com.ar/huevos?_q=huevos&...,/html/body/div[3]/div/div[1]/div/div[5]/div/di...
4,17,Dia,huevos,https://diaonline.supermercadosdia.com.ar/huev...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
5,18,Disco,huevos,https://www.disco.com.ar/huevos?_q=huevos&map=ft,/html/body/div[2]/div/div[1]/div/div[10]/div/d...
6,19,Carrefour,queso,https://www.carrefour.com.ar/Lacteos-y-product...,/html/body/div[3]/div/div[1]/div/div[5]/div/di...
7,20,Dia,queso,https://diaonline.supermercadosdia.com.ar/ques...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
8,21,Disco,queso,https://www.disco.com.ar/quesos?_q=quesos&map=ft,/html/body/div[2]/div/div[1]/div/div[10]/div/d...


### 4. Llamada a la funcion extract_text(id, url, by_type, identifier)

In [185]:
# Se extrae la data de la web
productos = []
# se recorre el dataframe df
for index, row in df.iterrows():
    # variables que se le pasan a la función
    id = row['id']
    url = row['link']
    xpath = row['xpath']
    
    result = extract_text(id, url, By.XPATH, xpath) 
    if result:  # Solo añadir si no es None
        productos.append(result)
    
    # Esperar 5 segundos antes de la siguiente iteración
    print("Esperando 1 segundos antes de la siguiente iteración...")
    # time.sleep(5)

# Imprimir el diccionario resultante
print(productos)
    

Navegando a la URL: https://www.carrefour.com.ar/Almacen/Pastas-secas?order=
Esperando el elemento con xpath y /html/body/div[3]/div/div[1]/div/div[5]/div/div/section/div[2]/div/div[3]/div/div[2]/div/div[3]/div/div/div/div[2]/div[14]/section/a/article/div/div[10]/div/div/div/div[1]/span/span/span
Elemento encontrado: <selenium.webdriver.remote.webelement.WebElement (session="05e34068d8cb966f2024b2f8f7a35f95", element="f.42BEFE76AEC1C74112C067500B122603.d.7E119C49906DB6BDDBA90F7E5402EA7B.e.256")>
Texto encontrado para 13: $ 975,00
Esperando 1 segundos antes de la siguiente iteración...
Navegando a la URL: https://diaonline.supermercadosdia.com.ar/fideos?_q=fideos&map=ft
Esperando el elemento con xpath y /html/body/div[2]/div/div[1]/div/div[3]/div/div/section/div[2]/div/div[3]/section/div/div[3]/div/div[3]/div/div/div/div/div[14]/section/a/article/div/div/div[5]/div/div/div[1]/span/span/span
Elemento encontrado: <selenium.webdriver.remote.webelement.WebElement (session="00cd6c868c99f5fe8

In [186]:
# Crear el DataFrame
df_precio = pd.DataFrame(productos, columns=['id', 'precio'])
df_precio

,id,precio
0,13,"$ 975,00"
1,14,"$ 1.390,00"
2,15,$1.350
3,16,"$ 1.990,00"
4,17,"$ 1.580,00"
5,18,$4.250
6,19,"$ 2.990,00"
7,20,"$ 7.550,00"
8,21,$8.999


In [187]:
# Hacer el merge de los DataFrames en función de la columna 'id'
df_final = pd.merge(df, df_precio, on='id')
# Nuevo orden de columnas
columnas_reordenadas = ['id', 'empresa', 'producto', 'precio', 'link', 'xpath']

# Crear un nuevo DataFrame con las columnas reordenadas
df_final = df_final[columnas_reordenadas]
df_final

,id,empresa,producto,precio,link,xpath
0,13,Carrefour,fideos,"$ 975,00",https://www.carrefour.com.ar/Almacen/Pastas-se...,/html/body/div[3]/div/div[1]/div/div[5]/div/di...
1,14,Dia,fideos,"$ 1.390,00",https://diaonline.supermercadosdia.com.ar/fide...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
2,15,Disco,fideos,$1.350,https://www.disco.com.ar/fideos?_q=fideos&map=ft,/html/body/div[2]/div/div[1]/div/div[10]/div/d...
3,16,Carrefour,huevos,"$ 1.990,00",https://www.carrefour.com.ar/huevos?_q=huevos&...,/html/body/div[3]/div/div[1]/div/div[5]/div/di...
4,17,Dia,huevos,"$ 1.580,00",https://diaonline.supermercadosdia.com.ar/huev...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
5,18,Disco,huevos,$4.250,https://www.disco.com.ar/huevos?_q=huevos&map=ft,/html/body/div[2]/div/div[1]/div/div[10]/div/d...
6,19,Carrefour,queso,"$ 2.990,00",https://www.carrefour.com.ar/Lacteos-y-product...,/html/body/div[3]/div/div[1]/div/div[5]/div/di...
7,20,Dia,queso,"$ 7.550,00",https://diaonline.supermercadosdia.com.ar/ques...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
8,21,Disco,queso,$8.999,https://www.disco.com.ar/quesos?_q=quesos&map=ft,/html/body/div[2]/div/div[1]/div/div[10]/div/d...


In [188]:
# limpiaza y cambio de tipo de datos de las columnas 
def clean_price(price):
    # Eliminar el símbolo de dólar y los puntos de los miles
    clean_price = price.replace('$', '').replace('.', '').replace(',', '.')
    # Convertir a float
    return int(float(clean_price))

# Aplicar la función al DataFrame
df_final['precio'] = df_final['precio'].apply(clean_price)
df_final['id'] = df_final['id'].astype(int)
df_final

,id,empresa,producto,precio,link,xpath
0,13,Carrefour,fideos,975,https://www.carrefour.com.ar/Almacen/Pastas-se...,/html/body/div[3]/div/div[1]/div/div[5]/div/di...
1,14,Dia,fideos,1390,https://diaonline.supermercadosdia.com.ar/fide...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
2,15,Disco,fideos,1350,https://www.disco.com.ar/fideos?_q=fideos&map=ft,/html/body/div[2]/div/div[1]/div/div[10]/div/d...
3,16,Carrefour,huevos,1990,https://www.carrefour.com.ar/huevos?_q=huevos&...,/html/body/div[3]/div/div[1]/div/div[5]/div/di...
4,17,Dia,huevos,1580,https://diaonline.supermercadosdia.com.ar/huev...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
5,18,Disco,huevos,4250,https://www.disco.com.ar/huevos?_q=huevos&map=ft,/html/body/div[2]/div/div[1]/div/div[10]/div/d...
6,19,Carrefour,queso,2990,https://www.carrefour.com.ar/Lacteos-y-product...,/html/body/div[3]/div/div[1]/div/div[5]/div/di...
7,20,Dia,queso,7550,https://diaonline.supermercadosdia.com.ar/ques...,/html/body/div[2]/div/div[1]/div/div[3]/div/di...
8,21,Disco,queso,8999,https://www.disco.com.ar/quesos?_q=quesos&map=ft,/html/body/div[2]/div/div[1]/div/div[10]/div/d...


### 5. Conexión a la base de datos y carga de datos

In [189]:
def conectar_mysql():
    """Función para conectar a MySQL"""
    try:
        conn = mysql.connector.connect(
            user='',
            password='',
            host='mysql-34ef87b0-leomar-b78e.aivencloud.com',
            database='price_data',
            port=14004  
        )
        print("Conexión a MySQL exitosa.")
        return conn
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_ACCESS_DENIED_ERROR:
            print("Error de acceso: Usuario o contraseña incorrectos.")
        elif err.errno == errorcode.ER_BAD_DB_ERROR:
            print("Error de base de datos: La base de datos no existe.")
        else:
            print(err)
        return None



In [190]:
def crear_tabla(conn):
    """Crea la tabla 'precios' si no existe"""
    cursor = conn.cursor()
    try:
        # cursor.execute("DROP TABLE IF EXISTS producto")
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS precios (
                id_precios INT AUTO_INCREMENT PRIMARY KEY,
                id INT,
                empresa VARCHAR(100),
                producto VARCHAR(100),  
                precio FLOAT,
                link VARCHAR(2000),
                xpath VARCHAR(2000),
                created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
            )
        """)
        conn.commit()
        print("Tabla 'precios' creada exitosamente.")
    except mysql.connector.Error as err:
        print("Error al crear la tabla:", err)
    finally:
        cursor.close()


In [191]:

def cargar_datos_db(conn, df):
    """Carga datos desde un DataFrame de pandas a la tabla 'precios' en MySQL"""
    cursor = conn.cursor()
    try:
        data = [tuple(row) for row in df.values]
        insert_query = """
            INSERT INTO precios
            (id, empresa, producto, precio, link, xpath)
            VALUES (%s, %s, %s, %s, %s, %s)
        """
        cursor.executemany(insert_query, data)
        conn.commit()
        print("Datos cargados en MySQL.")
    except mysql.connector.Error as err:
        print("Error al cargar datos en MySQL:", err)
        conn.rollback()
    finally:
        cursor.close()



In [192]:

conn = conectar_mysql()
if conn:
    crear_tabla(conn)
    df_final.columns = df_final.columns.str.lower()  # Asegúrate de que los nombres de las columnas coinciden con los nombres en la tabla
    print("DataFrame cargado")
    print(df_final.head())  # Imprime los primeros registros del DataFrame para verificar los datos
    cargar_datos_db(conn, df_final)
    conn.close()

Error de acceso: Usuario o contraseña incorrectos.
